In [280]:
import pandas as pd
import numpy as np
import hvplot
import matplotlib
import statsmodels.api as sm
import xlsxwriter
import math

#data
data = pd.read_csv('aussie_equities_scoring_python_15072024.csv', index_col=0, encoding='latin-1')
data1 = data.apply(lambda x: pd.to_numeric(x, errors='coerce'))
data2 = data1.copy()
data2.columns = [data1.columns, data.iloc[0]]
data1.columns = data.iloc[0]
data_cleaned = data1.iloc[1:]

#tracking error
excess_return = data_cleaned.iloc[:,5:].sub(data_cleaned.iloc[:,:1]['Market'], axis=0)
tracking_errors = excess_return.std() * math.sqrt(12)
tracking_errors.index = data2.iloc[:,5:].columns

#beta estimation
output = {}
output2 = {}
for i in data_cleaned.iloc[:,5:].columns:
    y = data_cleaned[i]
    x = data_cleaned.iloc[:,:5]
    if y.count() < 24:
            output[i] = np.nan
            output2[i] = np.nan
    else: 
        model = sm.OLS(y, sm.add_constant(x), missing='drop').fit()
        output[i] = model.params
        output2[i] = model.tvalues

coefficients = pd.DataFrame(output)
tstats = pd.DataFrame(output2)
coefficients.columns = data2.iloc[:,5:].columns
tstats.columns = data2.iloc[:,5:].columns

#classification
classifications = []
for i in tstats.columns:
    ranks = tstats[i].sort_values(axis=0, ascending = False)
    ranks_cleaned = ranks.drop(['Market','const'],axis=0)
    if ranks_cleaned.iloc[0] > 2.581:
        if not (ranks_cleaned.index[-1] == 'Value' and ranks_cleaned.loc['Value'] < -2.581 and (abs(ranks_cleaned.loc['Value']) > ranks_cleaned.iloc[0])):
            classifications.append(ranks_cleaned.index[0])
        else:
            classifications.append('Growth')

    elif (ranks_cleaned.loc['Value'] < -2.581 and (abs(ranks_cleaned.loc['Value']) > ranks_cleaned.iloc[0])):
        classifications.append('Growth')

    elif tracking_errors.loc[i] < 0.02:
        classifications.append('Beta')

    else:
        classifications.append('Alpha')
    

final_df = tstats.T
final_df['Classification'] = classifications
final_df['Tracking Error'] = tracking_errors

In [284]:
final_df.to_csv('new_scoring.csv')

In [265]:
#bennelong trial
a = tstats[(    'Bennelong Concentrated Australian Eq', 'BFL0002AU')].sort_values(axis=0, ascending = False).drop(['Market','const'],axis=0)
#a.index[-1] == 'Value' and a.iloc[-1] < -2.581 and (abs(a.iloc[-1]) >a.iloc[0])